<a href="https://colab.research.google.com/github/vinayaka16041999/Data-Science-ML-Projects/blob/master/HotelBooking_CaseStudy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# End-to-End ML


## Table of Contents


*   Data Description
*   Handling Missing Values
*   Handling Wrong Data
*Encoding the Categorical Columns (Label Encoding)
*Detecting and Removing Outliers using Isolation Forest Algorithm
*Feature Engineering using Boruta
*OneHotEncoding the Categorical Columns
*Normalizing the Numerical Columns to Common Range



In [1]:
import pandas as pd
import numpy as np
import seaborn as snb
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

## Data Description

In [2]:
df = pd.read_csv(r'/content/hotel_bookings.csv')
df.shape

(119390, 32)

In [3]:
df.head(10)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,7/1/2015
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,7/1/2015
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,7/2/2015
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,7/2/2015
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,7/3/2015
5,Resort Hotel,0,14,2015,July,27,1,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,7/3/2015
6,Resort Hotel,0,0,2015,July,27,1,0,2,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,0,No Deposit,NaN,NaN,0,Transient,107.0,0,0,Check-Out,7/3/2015
7,Resort Hotel,0,9,2015,July,27,1,0,2,2,0.0,0,FB,PRT,Direct,Direct,0,0,0,C,C,0,No Deposit,303.0,NaN,0,Transient,103.0,0,1,Check-Out,7/3/2015
8,Resort Hotel,1,85,2015,July,27,1,0,3,2,0.0,0,BB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,82.0,0,1,Canceled,5/6/2015
9,Resort Hotel,1,75,2015,July,27,1,0,3,2,0.0,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,D,D,0,No Deposit,15.0,NaN,0,Transient,105.5,0,0,Canceled,4/22/2015


In [23]:
df.describe()

,is_canceled,lead_time,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,agent,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests
count,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000
mean,0.370416,104.011416,0.927599,2.500302,1.856403,0.103886,0.007949,0.031912,0.087118,0.137097,0.221124,74.828319,2.321149,101.831122,0.062518,0.571363
std,0.482918,106.863097,0.998613,1.908286,0.579261,0.398555,0.097436,0.175767,0.844336,1.497437,0.652306,107.141953,17.594721,50.535790,0.245291,0.792798
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-6.380000,0.000000,0.000000
25%,0.000000,18.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000,0.000000,69.290000,0.000000,0.000000
50%,0.000000,69.000000,1.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.000000,0.000000,94.575000,0.000000,0.000000
75%,1.000000,160.000000,2.000000,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,152.000000,0.000000,126.000000,0.000000,1.000000
max,1.000000,737.000000,19.000000,50.000000,55.000000,10.000000,10.000000,1.000000,26.000000,72.000000,21.000000,535.000000,391.000000,5400.000000,8.000000,5.000000


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 26 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   stays_in_weekend_nights         119390 non-null  int64  
 4   stays_in_week_nights            119390 non-null  int64  
 5   adults                          119390 non-null  int64  
 6   children                        119390 non-null  float64
 7   babies                          119390 non-null  int64  
 8   meal                            119390 non-null  object 
 9   country                         119390 non-null  object 
 10  market_segment                  119390 non-null  object 
 11  distribution_channel            119390 non-null  object 
 12  is_repeated_gues

## Handling Missing Values

In [4]:
df.isna().sum()

hotel                                  0
is_canceled                            0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               4
babies                                 0
meal                                   0
country                              488
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit_type                           0
agent                              16340
company         

In [5]:
def handle_missing():
  temp = df
  temp.drop(columns=['company','arrival_date_year','arrival_date_month','arrival_date_week_number','arrival_date_day_of_month','reservation_status_date'],inplace=True)
  for col in temp.columns:
    if temp[col].isnull().sum()>0:
      temp.fillna(temp[col].median(),inplace=True)
  return temp

In [6]:
missing_dataless_df = handle_missing()
missing_dataless_df.isna().sum()

hotel                             0
is_canceled                       0
lead_time                         0
stays_in_weekend_nights           0
stays_in_week_nights              0
adults                            0
children                          0
babies                            0
meal                              0
country                           0
market_segment                    0
distribution_channel              0
is_repeated_guest                 0
previous_cancellations            0
previous_bookings_not_canceled    0
reserved_room_type                0
assigned_room_type                0
booking_changes                   0
deposit_type                      0
agent                             0
days_in_waiting_list              0
customer_type                     0
adr                               0
required_car_parking_spaces       0
total_of_special_requests         0
reservation_status                0
dtype: int64

## Handle Wrong Data

In [7]:
def handle_wrong_data():
  temp = missing_dataless_df
  temp = temp[~((temp['children']==0) & (temp['adults']==0) & (temp['babies']==0))]
  return temp

In [8]:
corrected_data_df = handle_wrong_data()
corrected_data_df

,hotel,is_canceled,lead_time,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status
0,Resort Hotel,0,342,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,0.0,0,Transient,0.00,0,0,Check-Out
1,Resort Hotel,0,737,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,0.0,0,Transient,0.00,0,0,Check-Out
2,Resort Hotel,0,7,0,1,1,0.0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,0.0,0,Transient,75.00,0,0,Check-Out
3,Resort Hotel,0,13,0,1,1,0.0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304.0,0,Transient,75.00,0,0,Check-Out
4,Resort Hotel,0,14,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,0,Transient,98.00,0,1,Check-Out
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,City Hotel,0,23,2,5,2,0.0,0,BB,BEL,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,394.0,0,Transient,96.14,0,0,Check-Out
119386,City Hotel,0,102,2,5,3,0.0,0,BB,FRA,Online TA,TA/TO,0,0,0,E,E,0,No Deposit,9.0,0,Transient,225.43,0,2,Check-Out
119387,City Hotel,0,34,2,5,2,0.0,0,BB,DEU,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,9.0,0,Transient,157.71,0,4,Check-Out
119388,City Hotel,0,109,2,5,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,89.0,0,Transient,104.40,0,0,Check-Out


## Encoding the Categorical Columns (Label Encoding)

In [9]:
from sklearn.preprocessing import LabelEncoder
cat_list = corrected_data_df.select_dtypes(include= ['object']).columns.tolist()
le = LabelEncoder()
label_encoded_df = corrected_data_df.copy()
for col in label_encoded_df.columns:
  if col in cat_list:
    label_encoded_df[col] = label_encoded_df[col].astype('category')
label_encoded_df[cat_list] = label_encoded_df[cat_list].apply(lambda x: x.cat.codes)
label_encoded_df.head()

,hotel,is_canceled,lead_time,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status
0,1,0,342,0,0,2,0.0,0,0,136,3,1,0,0,0,2,2,3,0,0.0,0,2,0.0,0,0,1
1,1,0,737,0,0,2,0.0,0,0,136,3,1,0,0,0,2,2,4,0,0.0,0,2,0.0,0,0,1
2,1,0,7,0,1,1,0.0,0,0,60,3,1,0,0,0,0,2,0,0,0.0,0,2,75.0,0,0,1
3,1,0,13,0,1,1,0.0,0,0,60,2,0,0,0,0,0,0,0,0,304.0,0,2,75.0,0,0,1
4,1,0,14,0,2,2,0.0,0,0,60,6,3,0,0,0,0,0,0,0,240.0,0,2,98.0,0,1,1


## Detecting and Removing Outliers using Isolation Forest Algorithm

In [10]:
from sklearn.ensemble import IsolationForest
out_detect = IsolationForest(max_samples=100, random_state=4, contamination='auto')
outlier = out_detect.fit_predict(label_encoded_df)
outlier_removed_df = label_encoded_df[np.where(outlier == 1, True, False)]

In [11]:
outlier_removed_df.shape

(102639, 26)

In [12]:
label_encoded_df.shape

(119210, 26)

In [13]:
y = outlier_removed_df['is_canceled']
x = outlier_removed_df.drop(columns='is_canceled')

## Feature Engineering using Boruta

In [14]:
!pip install Boruta
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy

In [15]:
selector = BorutaPy(RandomForestClassifier(),n_estimators='auto',verbose=2,random_state=2)
selector.fit(np.array(x),np.array(y))

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	25
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	25
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	25
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	25
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	25
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	25
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	25
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	16
Tentative: 	1
Rejected: 	8
Iteration: 	9 / 100
Confirmed: 	16
Tentative: 	1
Rejected: 	8
Iteration: 	10 / 100
Confirmed: 	16
Tentative: 	1
Rejected: 	8
Iteration: 	11 / 100
Confirmed: 	16
Tentative: 	1
Rejected: 	8
Iteration: 	12 / 100
Confirmed: 	16
Tentative: 	1
Rejected: 	8
Iteration: 	13 / 100
Confirmed: 	16
Tentative: 	1
Rejected: 	8
Iteration: 	14 / 100
Confirmed: 	16
Tentative: 	1
Rejected: 	8
Iteration: 	15 / 100
Confirmed: 	16
Tentative: 	1
Rejected: 	8
Iteration: 	16 / 100
Confirmed: 	16
Tentative: 	1
Rejected: 	8
I

BorutaPy(alpha=0.05,
         estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                          class_weight=None, criterion='gini',
                                          max_depth=None, max_features='auto',
                                          max_leaf_nodes=None, max_samples=None,
                                          min_impurity_decrease=0.0,
                                          min_impurity_split=None,
                                          min_samples_leaf=1,
                                          min_samples_split=2,
                                          min_weight_fraction_leaf=0.0,
                                          n_estimators=58, n_jobs=None,
                                          oob_score=False,
                                          random_state=RandomState(MT19937) at 0x7F3C18DFAE20,
                                          verbose=0, warm_start=False),
         max_iter=100, n_estimators=

In [16]:
important_columns = np.array(x.columns)[np.array(selector.support_)]

In [17]:
tot = important_columns
cat = ['country','market_segment','distribution_channel','assigned_room_type','deposit_type','agent','customer_type','reservation_status']
num = list(set(tot).difference(set(cat)))
num = list(set(num).difference(['is_canceled']))

In [18]:
feature_engineered_df = outlier_removed_df[important_columns]

## OneHotEncoding the Categorical Columns

In [19]:
def get_onehotencode():
  temp = feature_engineered_df
  temp = pd.get_dummies(temp,columns=cat,drop_first=True)
  return temp

In [20]:
one_hot_encoded_df = get_onehotencode()
one_hot_encoded_df.head()

,hotel,lead_time,stays_in_week_nights,previous_cancellations,booking_changes,adr,required_car_parking_spaces,total_of_special_requests,country_1,country_2,country_4,country_5,country_6,country_7,country_8,country_9,country_10,country_11,country_12,country_13,country_14,country_15,country_16,country_17,country_18,country_19,country_20,country_21,country_22,country_23,country_24,country_25,country_26,country_27,country_29,country_30,country_31,country_32,country_33,country_34,...,agent_432.0,agent_434.0,agent_436.0,agent_438.0,agent_441.0,agent_444.0,agent_446.0,agent_449.0,agent_450.0,agent_453.0,agent_454.0,agent_455.0,agent_459.0,agent_461.0,agent_464.0,agent_467.0,agent_468.0,agent_469.0,agent_474.0,agent_475.0,agent_476.0,agent_479.0,agent_480.0,agent_483.0,agent_484.0,agent_492.0,agent_493.0,agent_495.0,agent_502.0,agent_508.0,agent_509.0,agent_510.0,agent_526.0,agent_531.0,agent_535.0,customer_type_1,customer_type_2,customer_type_3,reservation_status_1,reservation_status_2
2,1,7,1,0,0,75.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
3,1,13,1,0,0,75.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
4,1,14,2,0,0,98.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
5,1,14,2,0,0,98.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
6,1,0,2,0,0,107.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0


## Normalizing the Numerical Columns to Common Range

In [21]:
from sklearn.preprocessing import StandardScaler
def scale_data():
  temp = one_hot_encoded_df
  sc = StandardScaler()
  temp[num] = sc.fit_transform(temp[num])
  return temp

In [22]:
scaled_df = scale_data()
scaled_df.head()

,hotel,lead_time,stays_in_week_nights,previous_cancellations,booking_changes,adr,required_car_parking_spaces,total_of_special_requests,country_1,country_2,country_4,country_5,country_6,country_7,country_8,country_9,country_10,country_11,country_12,country_13,country_14,country_15,country_16,country_17,country_18,country_19,country_20,country_21,country_22,country_23,country_24,country_25,country_26,country_27,country_29,country_30,country_31,country_32,country_33,country_34,...,agent_432.0,agent_434.0,agent_436.0,agent_438.0,agent_441.0,agent_444.0,agent_446.0,agent_449.0,agent_450.0,agent_453.0,agent_454.0,agent_455.0,agent_459.0,agent_461.0,agent_464.0,agent_467.0,agent_468.0,agent_469.0,agent_474.0,agent_475.0,agent_476.0,agent_479.0,agent_480.0,agent_483.0,agent_484.0,agent_492.0,agent_493.0,agent_495.0,agent_502.0,agent_508.0,agent_509.0,agent_510.0,agent_526.0,agent_531.0,agent_535.0,customer_type_1,customer_type_2,customer_type_3,reservation_status_1,reservation_status_2
2,1.633332,-0.915698,-0.846231,-0.111371,-0.302108,-0.549221,-0.189343,-0.721491,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
3,1.633332,-0.859658,-0.846231,-0.111371,-0.302108,-0.549221,-0.189343,-0.721491,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
4,1.633332,-0.850318,-0.222752,-0.111371,-0.302108,-0.031034,-0.189343,0.596344,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
5,1.633332,-0.850318,-0.222752,-0.111371,-0.302108,-0.031034,-0.189343,0.596344,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
6,1.633332,-0.981077,-0.222752,-0.111371,-0.302108,0.171734,-0.189343,-0.721491,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
